In [12]:
import PyPDF2
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import re

In [108]:
import requests
from bs4 import BeautifulSoup

r = requests.get('http://abbreviations.yourdictionary.com/articles/state-abbrev.html')
soup = BeautifulSoup(r.content, 'html.parser')
states = DataFrame((ele.text.split(' - ')[0], ele.text.split(' - ')[1]) for ele in soup.select('#article_main_content > ul > li')) \
    .reset_index() \
    .rename(columns={'index': 'id', 0: 'state', 1: 'abbreviation'}) \
    .assign(id=lambda x: x['id'] + 1)
states
states.to_csv('states.csv', index=False)

In [134]:
counties_raw = pd.read_csv('populations.csv')['Geography'] 
counties = DataFrame(counties_raw.str.split(', ').tolist(), columns=['county', 'state'])

counties = pd.merge(counties, states, left_on='state', right_on='state') \
    .drop(columns=['state', 'abbreviation']) \
    .reset_index() \
    .rename(columns={'id': 'state_id', 'index': 'id', 'county': 'county'}) \
    .assign(id=lambda x: x['id'] + 1)
    
counties.to_csv('counties.csv', index=False)

In [171]:
metro_raw = pd.read_csv('metro_counties.csv', encoding='ISO-8859-1', header=2)

metro_raw = metro_raw[['CBSA Title', 'Metropolitan/Micropolitan Statistical Area', 'County/County Equivalent', 'State Name']] \
    [(metro_raw['Metropolitan/Micropolitan Statistical Area'] == 'Metropolitan Statistical Area') & (metro_raw['State Name'] != 'Puerto Rico')] \
    .drop(columns=['Metropolitan/Micropolitan Statistical Area'])
metros = DataFrame(metro_raw['CBSA Title'].unique()) \
    .reset_index() \
    .rename(columns={'index': 'id', 0: 'metro'}) \
    .assign(id=lambda x: x['id'] + 1)
metros.to_csv('metros.csv', index=False)

In [186]:
metro_county = metro_raw \
    .rename(columns={'CBSA Title': 'metro', 'County/County Equivalent': 'county', 'State Name': 'state'}) \
    .merge(metros, left_on='metro', right_on='metro', ) \
    .rename(columns={'id': 'metro_id'}) \
    .drop(columns=['metro']) \
    .merge(states, left_on='state', right_on='state') \
    .rename(columns={'id': 'state_id'}) \
    .drop(columns=['state', 'abbreviation']) \
    .merge(counties, left_on=['county', 'state_id'], right_on=['county', 'state_id']) \
    .rename(columns={'id': 'county_id'}) \
    .drop(columns=['county', 'state_id']) \
    .reset_index() \
    .rename(columns={'index': 'id', 'metro_id': 'metro_id', 'county_id': 'county_id'}) \
    .assign(id=lambda x: x['id'] + 1)
metro_county.to_csv('metros_counties.csv', index=False)

In [223]:
populations_raw = pd.read_csv('populations.csv', encoding='ISO-8859-1', header=0) \
    .drop(columns=['Id', 'Id2']) \

populations_raw[['county', 'state']] = populations_raw['Geography'].str.split(', ', expand=True)
populations = populations_raw \
    .drop(columns=['Geography']) \
    .merge(states, left_on='state', right_on='state') \
    .drop(columns=['state', 'abbreviation']) \
    .merge(counties, left_on=['county', 'id'], right_on=['county', 'state_id']) \
    .drop(columns=['county', 'id_x', 'state_id']) \
    .rename(columns={'id_y': 'county_id'})
populations_complete = pd.melt(populations, id_vars=['county_id'], value_vars=['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']) \
    .rename(columns={'variable': 'year', 'value': 'population'}) \
    .sort_values('county_id') \
    .reset_index(drop=True) \
    .reset_index() \
    .rename(columns={'index': 'id'}) \
    .assign(id=lambda x: x['id'] + 1)
populations_complete.to_csv('populations_complete.csv', index=False)